# Aggregating and joining data

This is the second introductory tutorial to Ibis. If you are new to Ibis, you may want to start
by the [first tutorial](01-Introduction-to-Ibis.html).

In the first tutorial, we saw how to operate on the data of a table. We will work again with
the `countries` table as we did previously.

In [1]:
import os
import ibis


ibis.options.interactive = True

connection = ibis.sqlite.connect(os.path.join('data', 'geography.db'))
countries = connection.table('countries')

countries['name', 'continent', 'area_km2', 'population']

                     name continent   area_km2  population
0                 Andorra        EU      468.0       84000
1    United Arab Emirates        AS    82880.0     4975593
2             Afghanistan        AS   647500.0    29121286
3     Antigua and Barbuda        NA      443.0       86754
4                Anguilla        NA      102.0       13254
..                    ...       ...        ...         ...
247                 Yemen        AS   527970.0    23495361
248               Mayotte        AF      374.0      159042
249          South Africa        AF  1219912.0    49000000
250                Zambia        AF   752614.0    13460305
251              Zimbabwe        AF   390580.0    13061000

[252 rows x 4 columns]

## Expressions

We will continue by exploring the data by continent. We will start by creating an expression
with the continent names, since our table only contains the codes.

An expression is one or more operations performed over the data. They can be used to retrieve the
data, or to build more complex operations.

In this case we will use a `case` conditional statement, to replace values depending on a condition.

In [2]:
continent_name = (countries['continent'].case()
                                        .when('NA', 'North America')
                                        .when('SA', 'South America')
                                        .when('EU', 'Europe')
                                        .when('AF', 'Africa')
                                        .when('AS', 'Asia')
                                        .when('OC', 'Oceania')
                                        .when('AN', 'Anctartica')
                                        .end()
                                        .name('continent_name'))
continent_name

0             Europe
1               Asia
2               Asia
3      North America
4      North America
           ...      
247             Asia
248           Africa
249           Africa
250           Africa
251           Africa
Name: tmp, Length: 252, dtype: object

What we did is to take the values of the column `countries['continent']`, and we created a calculated
column with the names of the continents, as defined in the `when` methods.

This calculated column is an expression, meaning that is not stored in the database. Now we can query
again the table `countries`, but replacing the original `continent` column by our expression.

Note that the name of the column is not obtained by the name of the variable, but it was defined in the
expression with the `.name('continent_name')`.

In [3]:
countries['name', continent_name, 'area_km2', 'population']

                     name continent_name   area_km2  population
0                 Andorra         Europe      468.0       84000
1    United Arab Emirates           Asia    82880.0     4975593
2             Afghanistan           Asia   647500.0    29121286
3     Antigua and Barbuda  North America      443.0       86754
4                Anguilla  North America      102.0       13254
..                    ...            ...        ...         ...
247                 Yemen           Asia   527970.0    23495361
248               Mayotte         Africa      374.0      159042
249          South Africa         Africa  1219912.0    49000000
250                Zambia         Africa   752614.0    13460305
251              Zimbabwe         Africa   390580.0    13061000

[252 rows x 4 columns]

Just for illustration, let's repeat the same query, but renaming the expression to `continent`
when using it in the list of columns to fetch.

In [4]:
countries['name', continent_name.name('continent'), 'area_km2', 'population']

                     name      continent   area_km2  population
0                 Andorra         Europe      468.0       84000
1    United Arab Emirates           Asia    82880.0     4975593
2             Afghanistan           Asia   647500.0    29121286
3     Antigua and Barbuda  North America      443.0       86754
4                Anguilla  North America      102.0       13254
..                    ...            ...        ...         ...
247                 Yemen           Asia   527970.0    23495361
248               Mayotte         Africa      374.0      159042
249          South Africa         Africa  1219912.0    49000000
250                Zambia         Africa   752614.0    13460305
251              Zimbabwe         Africa   390580.0    13061000

[252 rows x 4 columns]

## Aggregating data

Now, let's group our data by continent, and let's find the total population of each.

In [5]:
countries.group_by(continent_name).aggregate(countries['population'].sum().name('total_population'))

  continent_name  total_population
0         Africa        1021238685
1     Anctartica               170
2           Asia        4130584841
3         Europe         750724554
4  North America         540204371
5        Oceania          36067549
6  South America         400143568

We can see how Asia is the most populated country, followed by Africa. And the Anctartica is the less populated,
as we could expect.

The code to aggregate has two main parts:
- The `group_by` method, that receive the column, expression or list of them to group by
- The `aggregate` method, that receives an expression with the reduction we want to apply

To make things a bit clearer, let's first save the reduction.

In [6]:
total_population = countries['population'].sum().name('total_population')
total_population

6878963738

As we can see, if we perform the operation directly, we will get the sum of the total in the column.

But if we take the expression as the parameter of the `aggregate` method, then, the total is computed
over every group defined by the `group_by` method.

In [7]:
countries.group_by(continent_name).aggregate(total_population)

  continent_name  total_population
0         Africa        1021238685
1     Anctartica               170
2           Asia        4130584841
3         Europe         750724554
4  North America         540204371
5        Oceania          36067549
6  South America         400143568

If we want to compute two aggregates at the same time, we can pass a list to the `aggregate` method.

For illustration, we use the `continent` column, instead of the `continent_names` expression. We can
use both column names and expressions, and also a list with any of them (e.g. `[continent_names, 'name']`.

In [8]:
countries.group_by('continent').aggregate([total_population,
                                           countries['area_km2'].mean().name('average_area')])

  continent  total_population  average_area
0        AF        1021238685  5.234534e+05
1        AN               170  2.802439e+06
2        AS        4130584841  6.196685e+05
3        EU         750724554  4.293017e+05
4        NA         540204371  5.836313e+05
5        OC          36067549  3.044157e+05
6        SA         400143568  1.272751e+06

## Joining data

Now we are going to get the total gross domestic product (GDP) for each continent. In this case, the GDP data
is not in the same table `countries`, but in a table `gdp`.

In [9]:
gdp = connection.table('gdp')
gdp

     country_code  year         value
0             ABW  1986  4.054634e+08
1             ABW  1987  4.876025e+08
2             ABW  1988  5.964236e+08
3             ABW  1989  6.953044e+08
4             ABW  1990  7.648871e+08
...           ...   ...           ...
9995          SVK  2002  3.513034e+10
9996          SVK  2003  4.681659e+10
9997          SVK  2004  5.733202e+10
9998          SVK  2005  6.278531e+10
9999          SVK  2006  7.070810e+10

[10000 rows x 3 columns]

Now, we are going to join this data, with the `countries` table, so we can obtain the continent
of each country. The `countries` table have different codes for the countries, let's find out which
is the one matching the three letter code in the `gdp` table.

In [10]:
countries

    iso_alpha2 iso_alpha3  iso_numeric fips                  name  \
0           AD        AND           20   AN               Andorra   
1           AE        ARE          784   AE  United Arab Emirates   
2           AF        AFG            4   AF           Afghanistan   
3           AG        ATG           28   AC   Antigua and Barbuda   
4           AI        AIA          660   AV              Anguilla   
..         ...        ...          ...  ...                   ...   
247         YE        YEM          887   YM                 Yemen   
248         YT        MYT          175   MF               Mayotte   
249         ZA        ZAF          710   SF          South Africa   
250         ZM        ZMB          894   ZA                Zambia   
251         ZW        ZWE          716   ZI              Zimbabwe   

              capital   area_km2  population continent  
0    Andorra la Vella      468.0       84000        EU  
1           Abu Dhabi    82880.0     4975593        AS  


The code in `gdp` corresponds to `iso_alpha2` in the `countries` table. We can also see,
how the `gdp` table has `247` rows, while `countries` has `252`. We will start joining the
two tables by the codes that match, discarding the codes that do not exist in both tables.
This is called an inner join.

In [11]:
countries_and_gdp = countries.inner_join(gdp,
                                         predicates=countries['iso_alpha3'] == gdp['country_code'])
countries_and_gdp.materialize()

     iso_alpha2 iso_alpha3  iso_numeric fips      name           capital  \
0            AD        AND           20   AN   Andorra  Andorra la Vella   
1            AD        AND           20   AN   Andorra  Andorra la Vella   
2            AD        AND           20   AN   Andorra  Andorra la Vella   
3            AD        AND           20   AN   Andorra  Andorra la Vella   
4            AD        AND           20   AN   Andorra  Andorra la Vella   
...         ...        ...          ...  ...       ...               ...   
9482         ZW        ZWE          716   ZI  Zimbabwe            Harare   
9483         ZW        ZWE          716   ZI  Zimbabwe            Harare   
9484         ZW        ZWE          716   ZI  Zimbabwe            Harare   
9485         ZW        ZWE          716   ZI  Zimbabwe            Harare   
9486         ZW        ZWE          716   ZI  Zimbabwe            Harare   

      area_km2  population continent country_code  year         value  
0        468.0 

What we did, is to call the `inner_join` method of the `countries` table, and passed as a parameter
the `gdp` table. The method receives a second parameter `predicates`, that is used to specify
how the join will be perform. In this case we want the `iso_alpha3` column in `countries`, to
match the `country_code` column in `gdp`. This is specified with the expression
`countries['iso_alpha3'] == gdp['country_code']`.

Finally, before we can use the values of the joined query, we need to materialize the result calling
its `materialize` method. Let's look with another example why joins need to be materialized.

We will now "clean up" the `countries` table before performing the join, by removing the columns we
don't want, and renaming the code column we are going to use.

In [12]:
countries_simplified = countries[countries['iso_alpha3'].name('country_code'), 'continent']
countries_simplified

    country_code continent
0            AND        EU
1            ARE        AS
2            AFG        AS
3            ATG        NA
4            AIA        NA
..           ...       ...
247          YEM        AS
248          MYT        AF
249          ZAF        AF
250          ZMB        AF
251          ZWE        AF

[252 rows x 2 columns]

Now, we can join again as we did before, but withe the expression `countries_simplified`, instead of with
the table `countries` directly.

In [13]:
countries_and_gdp = countries_simplified.inner_join(gdp,
                                                    predicates=countries_simplified['country_code'] == gdp['country_code'])
countries_and_gdp.materialize()

RelationError: Joined tables have overlapping names: ['country_code']

We are obtaining an exception, because the result would include the column `country_code` of
both tables. And duplicated column names are not allowed.

What we will do is to perform the join, but we won't yet materialize it.

In [14]:
countries_and_gdp = countries_simplified.inner_join(gdp,
                                                    predicates=countries_simplified['country_code'] == gdp['country_code'])

Now, we will select from the result, the columns that we are interested on, the continent, the value
of the GDP, and the country code just once. We can also rename columns before materializing. That would
be useful for example if `countries` also had a `value` column, and we wanted to get both, the one in
`gdp` and the one in `countries`.

In [15]:
gdp_with_continent = countries_and_gdp[countries_simplified['country_code'],
                                       countries_simplified['continent'],
                                       gdp['year'],
                                       gdp['value'].name('gdp')].materialize()
gdp_with_continent

     country_code continent  year           gdp
0             AND        EU  1970  7.861921e+07
1             AND        EU  1971  8.940982e+07
2             AND        EU  1972  1.134082e+08
3             AND        EU  1973  1.508201e+08
4             AND        EU  1974  1.865587e+08
...           ...       ...   ...           ...
9482          ZWE        AF  2013  1.909102e+10
9483          ZWE        AF  2014  1.949552e+10
9484          ZWE        AF  2015  1.996312e+10
9485          ZWE        AF  2016  2.054868e+10
9486          ZWE        AF  2017  2.281301e+10

[9487 rows x 4 columns]

In [16]:
countries_simplified

    country_code continent
0            AND        EU
1            ARE        AS
2            AFG        AS
3            ATG        NA
4            AIA        NA
..           ...       ...
247          YEM        AS
248          MYT        AF
249          ZAF        AF
250          ZMB        AF
251          ZWE        AF

[252 rows x 2 columns]

In [17]:
gdp

     country_code  year         value
0             ABW  1986  4.054634e+08
1             ABW  1987  4.876025e+08
2             ABW  1988  5.964236e+08
3             ABW  1989  6.953044e+08
4             ABW  1990  7.648871e+08
...           ...   ...           ...
9995          SVK  2002  3.513034e+10
9996          SVK  2003  4.681659e+10
9997          SVK  2004  5.733202e+10
9998          SVK  2005  6.278531e+10
9999          SVK  2006  7.070810e+10

[10000 rows x 3 columns]

In [18]:
countries_and_gdp = countries_and_gdp[countries_simplified, gdp]  # ['continent', gdp['country_code'].name('country_code'), 'value']

IntegrityError: Duplicate column name(s): ['country_code']

The table contains the GDP information per country, for a number of years, let's check which years.

In [19]:
gdp['year'].min(), gdp['year'].max()

(1960, 2017)

We are going to just take the information about the last available year, 2017.

In [20]:
gdp_2017 = gdp.filter(gdp['year'] == 2017)['country_code', 'value']
gdp_2017

    country_code         value
0            ABW  2.700559e+09
1            AFG  2.019176e+10
2            AGO  1.221238e+11
3            ALB  1.302506e+10
4            AND  3.013387e+09
..           ...           ...
242          XKX  7.227700e+09
243          YEM  2.681870e+10
244          ZAF  3.495541e+11
245          ZMB  2.586814e+10
246          ZWE  2.281301e+10

[247 rows x 2 columns]